In [15]:
# Setup for performance metrics - create nested directories
import random
from time import sleep
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 30
SIZES = [5, 10, 25, 50]


In [16]:
import sys
from pathlib import Path


# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *
from spytial.annotations import flag


In [ ]:
RED = "red"
BLACK = "black"



@orientation(selector='left & (RBNode -> (RBNode - NoneType.~key))', directions=['below', 'left'])
@orientation(selector='right & (RBNode -> (RBNode - NoneType.~key))', directions=['below', 'right'])
# Conception: Red / Black nodes are distinguished, (red and black are ~conception ~ convention)
@atomColor(
    selector='{ x : RBNode | @:(x.color) = red }',
    value='red'
)
@atomColor(
    selector='{ x : RBNode | @:(x.color) = black }',
    value='black'
)
# Convention: Nodes keys are visualized within the node.
@attribute(field="key")
@attribute(field="color")
@hideAtom(selector='{ x : RBNode | (x.key in NoneType) } + RBTree + int + NoneType + {s : str | @:s = red or @:s = black}') # Hide the NIL node
@hideField(field='parent') 
class RBNode:
    def __init__(self, key=None, color=BLACK, left=None, right=None, parent=None):
        self.key = key
        self.color = color
        self.left = left
        self.right = right
        self.parent = parent
    
    def __repr__(self):
        if self.key is None:
            return "()"
        left_str = "()" if self.left is NIL or self.left.key is None else repr(self.left)
        right_str = "()" if self.right is NIL or self.right.key is None else repr(self.right)
        return f"RBNode(key={self.key}, color={self.color!r}, left={left_str}, right={right_str})"

# Singleton NIL sentinel (all leaves point here)
NIL = RBNode(key=None, color=BLACK)
NIL.left = NIL.right = NIL.parent = NIL


class RBTree:
    def __init__(self):
        self.root = NIL

    # ----- Utility -----
    def search(self, key):
        x = self.root
        while x is not NIL and key != x.key:
            x = x.left if key < x.key else x.right
        return x

    def minimum(self, x):
        while x.left is not NIL:
            x = x.left
        return x

    # ----- Rotations (CLRS §13.2) -----
    def left_rotate(self, x):
        y = x.right
        assert y is not NIL, "left_rotate requires x.right != NIL"
        x.right = y.left
        if y.left is not NIL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is NIL:
            self.root = y
        elif x is x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, y):
        x = y.left
        assert x is not NIL, "right_rotate requires y.left != NIL"
        y.left = x.right
        if x.right is not NIL:
            x.right.parent = y
        x.parent = y.parent
        if y.parent is NIL:
            self.root = x
        elif y is y.parent.left:
            y.parent.left = x
        else:
            y.parent.right = x
        x.right = y
        y.parent = x

    # ----- Insert + Fixup (CLRS §13.3) -----
    def insert(self, key):
        z = RBNode(key=key, color=RED, left=NIL, right=NIL, parent=None)
        y = NIL
        x = self.root
        # BST insert
        while x is not NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is NIL:
            self.root = z
        elif z.key < y.key:
            y.left = z
        else:
            y.right = z
        # Fix red-black properties
        self._insert_fixup(z)
        return z

    def _insert_fixup(self, z):
        while z.parent.color is RED:
            if z.parent is z.parent.parent.left:
                y = z.parent.parent.right  # uncle
                if y.color is RED:
                    # Case 1
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z is z.parent.right:
                        # Case 2
                        z = z.parent
                        self.left_rotate(z)
                    # Case 3
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.right_rotate(z.parent.parent)
            else:
                # mirror image
                y = z.parent.parent.left
                if y.color is RED:
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z is z.parent.left:
                        z = z.parent
                        self.right_rotate(z)
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.left_rotate(z.parent.parent)
        self.root.color = BLACK

    # ----- (Optional) Traversal for testing -----
    def inorder(self, node=None, acc=None):
        if node is None:
            node, acc = self.root, []
        if node is NIL:
            return acc
        self.inorder(node.left, acc)
        acc.append((node.key, node.name))
        self.inorder(node.right, acc)
        return acc
    
    def __repr__(self):
        if self.root is NIL:
            return "RBTree(root=())"
        return f"RBTree(root={repr(self.root)})"
    
    


In [18]:
t = RBTree()
for k in [26, 17, 41, 47, 30, 38]:
    t.insert(k)

print(t)
diagram(t)

# s1 = t.root.right.left
# s1.left = s1.right
# diagram(t, method='browser') # Shows you each step of the insertion

RBTree(root=RBNode(26, 'black', RBNode(17, 'black', NIL, NIL), RBNode(41, 'red', RBNode(30, 'black', NIL, RBNode(38, ...)), RBNode(47, 'black', NIL, NIL))))
